## Agile Hardware Design
***
# Collections

<img src="./images/logo.svg" alt="agile hardware design logo" style="float:right"/>

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* Scala for & collections (only Seq for today)
* Chisel using collections
* Chisel Vec
* Chisel Mem

## Loading The Chisel Library Into a Notebook

In [23]:
// Before we start. We test for UCB stcture and make sure they worked.
// Below, we test for UCB stcture and make sure they worked.
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
println("path: "+path)

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/005_Collect/001_Code/source/load-ivy.sc


path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/005_Collect/001_Code/source/load-ivy.sc"

In [24]:
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/005_Collect/001_Code/Main.sc

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


Compiling /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/005_Collect/001_Code/Main.sc #2

In [25]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

In [26]:
// Test
class RegLand extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val en  = Input(Bool())
        val out = Output(Bool())
    })
    val r = Reg(Bool())
//    val r = RegInit(0.B)
    r := io.in
    io.out := r
//     io.out := RegNext(io.in, 0.B)
//     io.out := RegEnable(io.in, 0.B, io.en)
}
println (getVerilog(new RegLand))

Elaborating design...
Done elaborating.
module RegLand(
  input   clock,
  input   reset,
  input   io_in,
  input   io_en,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg  r; // @[cmd25.sc 7:16]
  assign io_out = r; // @[cmd25.sc 10:12]
  always @(posedge clock) begin
    r <= io_in; // @[cmd25.sc 9:7]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin 

defined class RegLand

## Scala `Seq`

* Scala's [collection library](https://docs.scala-lang.org/overviews/collections-2.13/overview.html) is one of its strengths
* [Seq](https://www.scala-lang.org/api/2.13.10/scala/collection/Seq.html) is an ordered collection (sequence), immutable by default
* Index into it with `()`
* A base class with multiple implementations
  * For many of our (small) cases, fine to use directly
  * Some style guides recommend specificity with `List` or `Vector`

In [27]:
val l = Seq(1,2,3)
l(1)
l.isEmpty
l.nonEmpty
l.length
Seq.fill(3)(8)
l.getClass

l: Seq[Int] = List(1, 2, 3)
res26_1: Int = 2
res26_2: Boolean = false
res26_3: Boolean = true
res26_4: Int = 3
res26_5: Seq[Int] = List(8, 8, 8)
res26_6: Class[T] = class scala.collection.immutable.$colon$colon

## Scala Ranges
* Can instantiate a range by `start until end` (exclusive)
* Can make range inclusive with `start to end`
* Can also provide a custom increment (default is 1) with `by`

In [28]:
0 until 4
0 to 3
0 to 4 by 2
3 to 0 by -1
0 to 10

res27_0: Range = Range(0, 1, 2, 3)
res27_1: Range.Inclusive = Range.Inclusive(0, 1, 2, 3)
res27_2: Range = Range.Inclusive(0, 2, 4)
res27_3: Range = Range.Inclusive(3, 2, 1, 0)
res27_4: Range.Inclusive = Range.Inclusive(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

## Scala `for`

* Iterates over iterable (typically range or collection)
* Sometimes use with `var` to pass results between iterations
* Also helpful for building up state space for testing
* Can also be used to do full [for comprehensions](https://docs.scala-lang.org/tour/for-comprehensions.html) (nested and filters)

In [29]:
println("Simple for loop:")
for (i <- 0 until 4)
    print(i + " ")

println (); println()
println("print with initial = -1:")
var last = -1
for (i <- 0 until 4) {
   print(last + " ")
   last = i
}

println (); println()
println("print with i and j")
for (i <- 0 until 6; 
     j <- i until 6  if (i + j == 6)) {
    println ("  i = " + i + " ")
    println ("  j = " + j + " ")
    println()
}

Simple for loop:
0 1 2 3 

print with initial = -1:
-1 0 1 2 

print with i and j
  i = 1 
  j = 5 

  i = 2 
  j = 4 

  i = 3 
  j = 3 



last: Int = 3

## Example: Delay Line with `for`

In [47]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n > 0)
    val regs = Seq.fill(n)(Reg(Bool()))
    regs(0) := io.in
    for (i <- 1 until n)
        regs(i) := regs(i-1)
    io.out := regs(n-1)
}
//println(getVerilog(new DelayNCycles(2)))

defined class DelayNCycles

### Test 2 Lines of Registers ###

In [46]:
println(getVerilog(new DelayNCycles(2)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
`endif // RANDOMIZE_REG_INIT
  reg  REG; // @[cmd31.sc 8:27]
  reg  lastConn; // @[cmd31.sc 8:27]
  assign io_out = lastConn; // @[cmd31.sc 9:12]
  always @(posedge clock) begin
    REG <= io_in; // @[cmd31.sc 8:27]
    lastConn <= REG; // @[cmd31.sc 8:27]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR

<img src="images/delayN.svg" alt="delay line schematic" style="width:30%;" align="left"/>

### Test for 8 lines of register ###

In [31]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n > 0)
    val regs = Seq.fill(n)(Reg(Bool()))
    regs(0) := io.in
    for (i <- 1 until n)
        regs(i) := regs(i-1)
    io.out := regs(n-1)
}
println(getVerilog(new DelayNCycles(8)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
  reg [31:0] _RAND_2;
  reg [31:0] _RAND_3;
  reg [31:0] _RAND_4;
  reg [31:0] _RAND_5;
  reg [31:0] _RAND_6;
  reg [31:0] _RAND_7;
`endif // RANDOMIZE_REG_INIT
  reg  regs_0; // @[cmd30.sc 7:31]
  reg  regs_1; // @[cmd30.sc 7:31]
  reg  regs_2; // @[cmd30.sc 7:31]
  reg  regs_3; // @[cmd30.sc 7:31]
  reg  regs_4; // @[cmd30.sc 7:31]
  reg  regs_5; // @[cmd30.sc 7:31]
  reg  regs_6; // @[cmd30.sc 7:31]
  reg  regs_7; // @[cmd30.sc 7:31]
  assign io_out = regs_7; // @[cmd30.sc 11:12]
  always @(posedge clock) begin
    regs_0 <= io_in; // @[cmd30.sc 8:13]
    regs_1 <= regs_0; // @[cmd30.sc 10:17]
    regs_2 <= regs_1; // @[cmd30.sc 10:17]
    regs_3 <= regs_2; // @[cmd30.sc 10:17]
    regs_4 <= regs_3; // @[cmd30.sc 10:17]
    regs_5 <= regs_4; // @[cmd30.sc 10:17]
    regs_6 <= regs_5; 

defined class DelayNCycles

## Example: Delay Line with `for` and `var`

* Judicious use of `var` can sometimes tidy things up
* Later, will cover functional programming techniques to also clean up

In [32]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    var lastConn = io.in
    for (i <- 0 until n)
        lastConn = RegNext(lastConn)
    io.out := lastConn
}

defined class DelayNCycles

In [33]:
//printVerilog(new DelayNCycles(3))
println (getVerilog(new DelayNCycles(3)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
  reg [31:0] _RAND_2;
`endif // RANDOMIZE_REG_INIT
  reg  REG; // @[cmd31.sc 8:27]
  reg  REG_1; // @[cmd31.sc 8:27]
  reg  lastConn; // @[cmd31.sc 8:27]
  assign io_out = lastConn; // @[cmd31.sc 9:12]
  always @(posedge clock) begin
    REG <= io_in; // @[cmd31.sc 8:27]
    REG_1 <= REG; // @[cmd31.sc 8:27]
    lastConn <= REG_1; // @[cmd31.sc 8:27]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial beg

## Using `for` in a Tester

In [48]:
class CombLogic extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Input(Bool())
        val out = Output(Bool())
    })
    io.out := (io.a && io.b) || ~io.c
    printf(p"a: ${io.a}, b: ${io.b}, c: ${io.c}, out: ${io.out}\n")
}

defined class CombLogic

In [53]:
test(new CombLogic) { dut =>
    for (a <- Seq(true, false)) {
        for (b <- Seq(true, false)) {
             for (c <- Seq(true, false)) {
                dut.io.a.poke(a.B)
                dut.io.b.poke(b.B)
                dut.io.c.poke(c.B)
                println(s"$a, $b, $c")
                val expected = (a && b) || !c
                dut.io.out.expect(expected.B)
                dut.clock.step()
             }
        }
    }
}

Elaborating design...
Done elaborating.
true, true, true
a:  1, b:  1, c:  1, out:  1
true, true, false
a:  1, b:  1, c:  0, out:  1
true, false, true
a:  1, b:  0, c:  1, out:  0
true, false, false
a:  1, b:  0, c:  0, out:  1
false, true, true
a:  0, b:  1, c:  1, out:  0
false, true, false
a:  0, b:  1, c:  0, out:  1
false, false, true
a:  0, b:  0, c:  1, out:  0
false, false, false
a:  0, b:  0, c:  0, out:  1
a:  0, b:  0, c:  0, out:  1
test CombLogic Success: 0 tests passed in 10 cycles in 0.015848 seconds 630.98 Hz


### Test with one for loop ###

In [54]:
class CombLogic extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Input(Bool())
        val out = Output(Bool())
    })
    io.out := (io.a && io.b) || ~io.c
    printf(p"a: ${io.a}, b: ${io.b}, c: ${io.c}, out: ${io.out}\n")
}

test(new CombLogic) { dut =>
    for (a <- Seq(true, false);
         b <- Seq(true, false);
         c <- Seq(true, false)) {
            dut.io.a.poke(a.B)
            dut.io.b.poke(b.B)
            dut.io.c.poke(c.B)
            println(s"$a, $b, $c")
            val expected = (a && b) || !c
            dut.io.out.expect(expected.B)
            dut.clock.step()
    }
}

Elaborating design...
Done elaborating.
true, true, true
a:  1, b:  1, c:  1, out:  1
true, true, false
a:  1, b:  1, c:  0, out:  1
true, false, true
a:  1, b:  0, c:  1, out:  0
true, false, false
a:  1, b:  0, c:  0, out:  1
false, true, true
a:  0, b:  1, c:  1, out:  0
false, true, false
a:  0, b:  1, c:  0, out:  1
false, false, true
a:  0, b:  0, c:  1, out:  0
false, false, false
a:  0, b:  0, c:  0, out:  1
a:  0, b:  0, c:  0, out:  1
test CombLogic Success: 0 tests passed in 10 cycles in 0.013690 seconds 730.44 Hz


defined class CombLogic

## Chisel `Vec` ##

* Chisel collection construct have 2 uses:
  * _Dynamic select_ in hardware (muxes)
  * Paramterize number of ports
* `Vec(num_elements, type)`
* Most often will want a Scala collection for addressing during elaboration
* `Vec` is a type, not a physical thing
* Use `Reg` of `Vec` for state
  * `Vec` of `Reg` not possible

```scala
Reg(Vec(num_elements, type))
```
* Can also use with `Wire`

```scala
Wire(Vec(num_elements, type))
```

In [55]:
class MyMuxN(n: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(w.W)))
        val sel = Input(UInt(log2Ceil(n).W))
        val out = Output(UInt(w.W))
    })
    io.out := io.in(io.sel)
}
//printVerilog(new MyMuxN(4,1))
println (getVerilog(new MyMuxN(4,1)))

Elaborating design...
Done elaborating.
module MyMuxN(
  input        clock,
  input        reset,
  input        io_in_0,
  input        io_in_1,
  input        io_in_2,
  input        io_in_3,
  input  [1:0] io_sel,
  output       io_out
);
  wire  _GEN_1 = 2'h1 == io_sel ? io_in_1 : io_in_0; // @[cmd54.sc 7:12 cmd54.sc 7:12]
  wire  _GEN_2 = 2'h2 == io_sel ? io_in_2 : _GEN_1; // @[cmd54.sc 7:12 cmd54.sc 7:12]
  assign io_out = 2'h3 == io_sel ? io_in_3 : _GEN_2; // @[cmd54.sc 7:12 cmd54.sc 7:12]
endmodule



defined class MyMuxN

## Using `Vec` to Parameterize # of Ports

In [37]:
class Reducer(n: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(w.W)))
// like val in  = Input(UInt((n*w).W))
        val out = Output(UInt(w.W))
    })
    require(n > 0)
    var totalSoFar = io.in(0)
    for (i <- 1 until n)
        totalSoFar = io.in(i) + totalSoFar
    io.out := totalSoFar
}
//printVerilog(new Reducer(2,2))
println (getVerilog(new Reducer(2,2)))

Elaborating design...
Done elaborating.
module Reducer(
  input        clock,
  input        reset,
  input  [1:0] io_in_0,
  input  [1:0] io_in_1,
  output [1:0] io_out
);
  assign io_out = io_in_1 + io_in_0; // @[cmd36.sc 10:31]
endmodule



defined class Reducer

<img src="images/reducer.svg" alt="reducer schematic" style="width:30%;" align="left"/>

## Read-Only Memory (ROM) with `VecInit`
* `VecInit` will create a `Wire` with its argument
* Can also use `VecInit` to initialize registers (with `RegInit`)

In [38]:
class DivByXTable(x: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(4.W))
        val out = Output(Bool())
    })
    var results = Seq[Bool]()
    for (i <- 0 to 15) {
        results = results :+ (i % x == 0).B
    }
    val table = VecInit(results)
    io.out := table(io.in)
}

defined class DivByXTable

In [39]:
//printVerilog(new DivByXTable(3))
println (getVerilog(new DivByXTable(3)))

Elaborating design...
Done elaborating.
module DivByXTable(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output       io_out
);
  wire  _GEN_1 = 4'h1 == io_in ? 1'h0 : 1'h1; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_2 = 4'h2 == io_in ? 1'h0 : _GEN_1; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_4 = 4'h4 == io_in ? 1'h0 : 4'h3 == io_in | _GEN_2; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_5 = 4'h5 == io_in ? 1'h0 : _GEN_4; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_7 = 4'h7 == io_in ? 1'h0 : 4'h6 == io_in | _GEN_5; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_8 = 4'h8 == io_in ? 1'h0 : _GEN_7; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_10 = 4'ha == io_in ? 1'h0 : 4'h9 == io_in | _GEN_8; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_11 = 4'hb == io_in ? 1'h0 : _GEN_10; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_13 = 4'hd == io_in ? 1'h0 : 4'hc == io_in | _GEN_11; // @[cmd37.sc 11:12 cmd37.sc 11:12]
  wire  _GEN_14 = 4'he == 

## Chisel `Mem`

* Construct for [memory](https://www.chisel-lang.org/chisel3/docs/explanations/memories.html) (dynamically addressable & mutable)
* Backend will choose appropriate implementation technology
* Default (Mem): _combinational read_ (0 cycle delay), _synchronous write_ (1 cycle delay)
  * Can tweak delay parameters
  * `SyncReadMem` has 1 cycle read delay
* Memory ports can be declared implicitly or explicitly
* Also has support for write masks

## Example: Register File (2R, 1W) - implicit ports

In [59]:
class RegFile() extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(5.W))
        val r1addr = Input(UInt(5.W))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val r0out =  Output(UInt(64.W))
        val r1out =  Output(UInt(64.W))
    })
    //val regs = Mem(32, UInt(64.W))
    val regs = Reg(Vec(32, UInt(64.W)))
    io.r0out := regs(io.r0addr)
    io.r1out := regs(io.r1addr)
    when(io.w0en) {
        regs(io.w0addr) := io.w0data
    }
}

defined class RegFile

In [60]:
//printVerilog(new RegFile())
println (getVerilog(new RegFile()))

Elaborating design...
Done elaborating.
module RegFile(
  input         clock,
  input         reset,
  input  [4:0]  io_r0addr,
  input  [4:0]  io_r1addr,
  input  [4:0]  io_w0addr,
  input         io_w0en,
  input  [63:0] io_w0data,
  output [63:0] io_r0out,
  output [63:0] io_r1out
);
`ifdef RANDOMIZE_REG_INIT
  reg [63:0] _RAND_0;
  reg [63:0] _RAND_1;
  reg [63:0] _RAND_2;
  reg [63:0] _RAND_3;
  reg [63:0] _RAND_4;
  reg [63:0] _RAND_5;
  reg [63:0] _RAND_6;
  reg [63:0] _RAND_7;
  reg [63:0] _RAND_8;
  reg [63:0] _RAND_9;
  reg [63:0] _RAND_10;
  reg [63:0] _RAND_11;
  reg [63:0] _RAND_12;
  reg [63:0] _RAND_13;
  reg [63:0] _RAND_14;
  reg [63:0] _RAND_15;
  reg [63:0] _RAND_16;
  reg [63:0] _RAND_17;
  reg [63:0] _RAND_18;
  reg [63:0] _RAND_19;
  reg [63:0] _RAND_20;
  reg [63:0] _RAND_21;
  reg [63:0] _RAND_22;
  reg [63:0] _RAND_23;
  reg [63:0] _RAND_24;
  reg [63:0] _RAND_25;
  reg [63:0] _RAND_26;
  reg [63:0] _RAND_27;
  reg [63:0] _RAND_28;
  reg [63:0] _RAND_29;
  reg

## Example: Register File (2R, 1W) - explicit ports

In [61]:
class RegFile extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(5.W))
        val r1addr = Input(UInt(5.W))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val r0out =  Output(UInt(64.W))
        val r1out =  Output(UInt(64.W))
    })
    val regs = Mem(32, UInt(64.W))
    io.r0out := regs.read(io.r0addr)
    io.r1out := regs.read(io.r1addr)
    when (io.w0en) {
        regs.write(io.w0addr,io.w0data)
    }
}

defined class RegFile

In [62]:
//printVerilog(new RegFile())
println (getVerilog(new RegFile()))

Elaborating design...
Done elaborating.
module RegFile(
  input         clock,
  input         reset,
  input  [4:0]  io_r0addr,
  input  [4:0]  io_r1addr,
  input  [4:0]  io_w0addr,
  input         io_w0en,
  input  [63:0] io_w0data,
  output [63:0] io_r0out,
  output [63:0] io_r1out
);
`ifdef RANDOMIZE_MEM_INIT
  reg [63:0] _RAND_0;
`endif // RANDOMIZE_MEM_INIT
  reg [63:0] regs [0:31]; // @[cmd60.sc 11:19]
  wire [63:0] regs_MPORT_data; // @[cmd60.sc 11:19]
  wire [4:0] regs_MPORT_addr; // @[cmd60.sc 11:19]
  wire [63:0] regs_MPORT_1_data; // @[cmd60.sc 11:19]
  wire [4:0] regs_MPORT_1_addr; // @[cmd60.sc 11:19]
  wire [63:0] regs_MPORT_2_data; // @[cmd60.sc 11:19]
  wire [4:0] regs_MPORT_2_addr; // @[cmd60.sc 11:19]
  wire  regs_MPORT_2_mask; // @[cmd60.sc 11:19]
  wire  regs_MPORT_2_en; // @[cmd60.sc 11:19]
  assign regs_MPORT_addr = io_r0addr;
  assign regs_MPORT_data = regs[regs_MPORT_addr]; // @[cmd60.sc 11:19]
  assign regs_MPORT_1_addr = io_r1addr;
  assign regs_MPORT_1_data 

## Example: Register File (NR, 1W) - parameterized ports

In [63]:
class RegFile(nRead: Int) extends Module {
    val io = IO(new Bundle {
        val raddr  = Input(Vec(nRead, UInt(5.W)))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val rout = Output(Vec(nRead, UInt(64.W)))
    })
    val regs = Mem(32, UInt(64.W))
    for (i <- 0 until nRead)
        io.rout(i) := regs(io.raddr(i))
    when(io.w0en) {
        regs(io.w0addr) := io.w0data
    }
}

defined class RegFile

In [64]:
//printVerilog(new RegFile(4))
println (getVerilog(new RegFile(4)))

Elaborating design...
Done elaborating.
module RegFile(
  input         clock,
  input         reset,
  input  [4:0]  io_raddr_0,
  input  [4:0]  io_raddr_1,
  input  [4:0]  io_raddr_2,
  input  [4:0]  io_raddr_3,
  input  [4:0]  io_w0addr,
  input         io_w0en,
  input  [63:0] io_w0data,
  output [63:0] io_rout_0,
  output [63:0] io_rout_1,
  output [63:0] io_rout_2,
  output [63:0] io_rout_3
);
`ifdef RANDOMIZE_MEM_INIT
  reg [63:0] _RAND_0;
`endif // RANDOMIZE_MEM_INIT
  reg [63:0] regs [0:31]; // @[cmd62.sc 9:19]
  wire [63:0] regs_MPORT_data; // @[cmd62.sc 9:19]
  wire [4:0] regs_MPORT_addr; // @[cmd62.sc 9:19]
  wire [63:0] regs_MPORT_1_data; // @[cmd62.sc 9:19]
  wire [4:0] regs_MPORT_1_addr; // @[cmd62.sc 9:19]
  wire [63:0] regs_MPORT_2_data; // @[cmd62.sc 9:19]
  wire [4:0] regs_MPORT_2_addr; // @[cmd62.sc 9:19]
  wire [63:0] regs_MPORT_3_data; // @[cmd62.sc 9:19]
  wire [4:0] regs_MPORT_3_addr; // @[cmd62.sc 9:19]
  wire [63:0] regs_MPORT_4_data; // @[cmd62.sc 9:19]
  wir

### Summary - When to Use Scala or Chisel Collection?

* You will want to use a Scala collection (probably `Seq`) most often
    * In a generator, want to instantiate N things
    * You may need to address which thing you want, but you are accessing it during hardware generation time
* You will need Chisel collections (`Vec`, `Mem`) when ...
    * You want the generated hardware to dynamically address components (put muxes into actual hardware)
    * You definitely want a memory (e.g. `SyncReadMem`)